In [1]:
import numpy as np
import pandas as pd

In [4]:
import random


parametros = {"perfil_1":{"media_1":5000,"desviacion_1":3000,
                         "media_2":5000,"desviacion_2":3000,
                         "media_3":5000,"desviacion_3":3000},
             "perfil_2":{"media_1":1000,"desviacion_1":3000,
                         "media_2":1000,"desviacion_2":3000,
                         "media_3":1000,"desviacion_3":3000},
              "perfil_3":{"media_1":9000,"desviacion_1":3000,
                         "media_2":9000,"desviacion_2":3000,
                         "media_3":9000,"desviacion_3":3000}
             }
perfiles = ["perfil_1","perfil_2","perfil_3"]
observaciones = []
perfil_obs = []
for idx in range(1000):
    perfil = random.sample(perfiles,1)
    perfil = perfil[0]
    perfil_obs.append(perfil)
    obs_1 = random.normalvariate(parametros[perfil]["media_1"], parametros[perfil]["desviacion_1"])
    obs_2 = random.normalvariate(parametros[perfil]["media_2"], parametros[perfil]["desviacion_2"])
    obs_3 = random.normalvariate(parametros[perfil]["media_3"], parametros[perfil]["desviacion_3"])

    observaciones.append([obs_1, obs_2, obs_3])

def conecta_vecinos_graph(vecinos,observaciones):
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    X = np.array(observaciones)
    nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    vecinos_cercanos_graph = {}
    for index in indices:
        vecinos_cercanos_graph[index[0]] = set(index[1:])

    return vecinos_cercanos_graph

graph_vecinos = conecta_vecinos_graph(5,observaciones)

def bfs(grafo, nodo0):
    visitado, cola = set(), [nodo0]
    while cola:
        vertex = cola.pop(0)
        if vertex not in visitado:
            visitado.add(vertex)
            if vertex not in grafo:
                grafo[vertex] = set()
                #print 'Tupla:', vertex, ',', len(grafo[vertex]), ',', len(visitado)
            cola.extend(grafo[vertex] - visitado)

    return visitado

def searchKNN(grafo, nodo0):
    klts = []
    faltantes = grafo
    visitados = bfs(grafo, nodo0)
    c=0
    while len(faltantes)>0:
        kls = {}

        for visitado in list(visitados):
            if grafo.has_key(visitado):
                kls[visitado]= grafo[visitado] #guardamos en el cluster
                del faltantes[visitado]
                visitados.remove(visitado)


        klts.append(kls)


        if len(faltantes) >0:
            #print faltantes.keys()[0]
            #print faltantes
            visitados = bfs(faltantes, faltantes.keys()[0])
    return klts

In [5]:
graph_vecinos = conecta_vecinos_graph(8, observaciones)
cluster=searchKNN(graph_vecinos, 0)

In [6]:
cluster

[{0: {68, 87, 193, 498, 615, 864, 925},
  1: {56, 72, 191, 204, 459, 492, 603},
  2: {10, 49, 303, 600, 885, 896, 964},
  3: {57, 252, 267, 357, 448, 980, 989},
  4: {507, 606, 641, 661, 827, 924, 973},
  5: {73, 105, 208, 309, 682, 875, 971},
  6: {122, 275, 407, 434, 451, 513, 919},
  7: {126, 495, 545, 571, 592, 663, 890},
  8: {130, 251, 296, 418, 582, 619, 995},
  9: {53, 289, 360, 493, 793, 823, 832},
  10: {2, 12, 49, 600, 634, 896, 990},
  11: {23, 240, 466, 481, 548, 561, 562},
  12: {10, 51, 231, 314, 600, 665, 700},
  13: {48, 136, 171, 277, 454, 777, 848},
  14: {113, 141, 217, 274, 469, 489, 788},
  15: {183, 468, 518, 553, 838, 936, 983},
  16: {24, 201, 229, 432, 692, 869, 873},
  17: {26, 269, 306, 665, 697, 700, 943},
  18: {33, 103, 223, 416, 499, 706, 738},
  19: {41, 184, 426, 444, 528, 630, 944},
  20: {32, 59, 237, 471, 729, 854, 960},
  21: {30, 140, 514, 518, 720, 760, 838},
  22: {123, 220, 231, 413, 494, 538, 669},
  23: {11, 240, 466, 481, 561, 562, 727},
  2

In [7]:
adjMat = np.array([[0L for j in range(len(cluster))]for i in range(len(cluster))])
for idA,A in enumerate(cluster):    
    for idB in xrange(idA+1,len(cluster)):
        #print idA,idB
        B = cluster[idB]
        inter = set(A.keys()).intersection(set(B.keys()))
        if inter!=set():
            print inter
            adjMat[idA,idB]=len(inter)
            

set([961, 387, 328, 713, 618, 84, 89])
set([101, 397, 976, 436, 86, 635, 829])
set([739, 361, 465, 660, 596, 349, 735])
set([834, 547, 516, 999, 944, 19, 444])
set([162, 940, 371, 820, 794, 766, 767])
set([584, 809, 620, 660, 671, 478, 159])
set([801, 290, 100, 69, 140, 149, 473])
set([160, 962, 235, 838, 678, 299, 674, 21, 763])
set([578, 195, 234, 748, 45, 820, 662, 500])
set([359, 169, 877, 943, 80, 119, 856])
set([679, 650, 371, 117, 921, 794, 981])
set([660])
set([820])
set([794, 371])


In [8]:
mmi =pd.DataFrame(adjMat)

In [9]:
mmi

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,7,7,7,7,7,7,7,9,8,7,7
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,0,2
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0
